In [ ]:
#import necessary packages

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn
from scipy import linalg
plt.style.use('seaborn-whitegrid')
from gatspy import datasets, periodic
import pywt
import gc
from sklearn.decomposition import PCA
from scipy.interpolate import UnivariateSpline

In [ ]:
#create pandas dataframe with training set time-series and metadata

train_series = pd.read_csv('training_set.csv')
train_series = train_series.sort_values(['object_id', 'passband'], ascending = [True, True])

train_metadata = pd.read_csv('training_set_metadata.csv')

train_series.head()

#print('Classes contained in the data set:')
types = train_metadata['target'].unique()
#print(types)

In [ ]:
#combine time series data and merge with metadata

#create arrays for object id and passband through which to iterate
ids = train_metadata['object_id'].values
passbands = [0, 1, 2, 3, 4, 5]

#append mjd, flux, and flux_err values per passband per object to the metadata
for band in passbands:
    mjd_column = []
    flux_column = []
    fluxerr_column = []
    for obj in ids:
        selection = train_series.loc[(train_series['object_id'] == obj) & (train_series['passband'] == band)]
        mjd_column.append(np.array(selection['mjd']))
        flux_column.append(np.array(selection['flux']))
        fluxerr_column.append(np.array(selection['flux_err']))
    train_metadata['mjd_passband_{}'.format(band)] = mjd_column
    train_metadata['flux_passband_{}'.format(band)] = flux_column
    train_metadata['fluxerr_passband_{}'.format(band)] = fluxerr_column

In [ ]:
#deleting unneccessary columns for the sake of efficiency
#print(train_metadata.columns.values)

del train_metadata['ra'], train_metadata['decl'], train_metadata['gal_l'], train_metadata['gal_b'], train_metadata['mwebv']

train_metadata.head()

In [ ]:
#plot distribution of object frequency in each class

train_metadata['target'].value_counts().plot(kind='bar')
plt.title('Training Set Class Frequencies')
plt.xlabel('Class')
plt.ylabel('Number of representative objects')

In [ ]:
#plot object distributions between survey fields
#ddf == 1 means the object comes from the ddf survey area
#ddf == 0 means the object comes from the wfd survey area
#ddf == 1 fluxes have significantly smaller uncertainties

#distribution of classes surveyed in ddf
plt.figure()
ddf_events = train_metadata[(train_metadata['ddf'] == 1)]
ddf_events['target'].value_counts().plot(kind='bar')

plt.title('Training Set Class Frequencies (DDF)')
plt.xlabel('Class')
plt.ylabel('Number of representative objects')

#distribution of classes surveyed in wfd
plt.figure()
wfd_events = train_metadata[(train_metadata['ddf'] == 0)]
wfd_events['target'].value_counts().plot(kind='bar')

plt.title('Training Set Class Frequencies (WFD)')
plt.xlabel('Class')
plt.ylabel('Number of representative objects')

WHAT CAN THIS DISTRIBUTION TELL US

In [ ]:
#create dataframes for each class represented
type_90 = train_metadata.loc[train_metadata['target'] == 90]
type_42 = train_metadata.loc[train_metadata['target'] == 42]
type_65 = train_metadata.loc[train_metadata['target'] == 65]
type_16 = train_metadata.loc[train_metadata['target'] == 16]
type_15 = train_metadata.loc[train_metadata['target'] == 15]
type_62 = train_metadata.loc[train_metadata['target'] == 62]
type_88 = train_metadata.loc[train_metadata['target'] == 88]
type_92 = train_metadata.loc[train_metadata['target'] == 92]
type_67 = train_metadata.loc[train_metadata['target'] == 67]
type_52 = train_metadata.loc[train_metadata['target'] == 52]
type_95 = train_metadata.loc[train_metadata['target'] == 95]
type_6 = train_metadata.loc[train_metadata['target'] == 6]
type_64 = train_metadata.loc[train_metadata['target'] == 64]
type_53 = train_metadata.loc[train_metadata['target'] == 53]

target_dataframes = [type_90, type_42, type_65, type_16,
                    type_15, type_62, type_88, type_92, type_67,
                    type_52, type_95, type_6, type_64, type_53]

type_90.head()

In [ ]:
#box and whisker plots for spectroscopic
#and photometric redshift values across classes

train_metadata['hostgal_specz']
train_metadata['hostgal_photoz']

spec_rs = [x['hostgal_specz'] for x in target_dataframes]
photo_rs = [x['hostgal_photoz'] for x in target_dataframes]
photerr_rs = [x['hostgal_photoz_err'] for x in target_dataframes]
labels = [90, 42, 65, 16, 15, 62, 88, 92, 67, 52, 95, 6, 64, 53]

plt.figure()
plt.title('Spectroscopic Redshift Across Classes')
plt.ylabel('Redshift')
plt.xlabel('Class')
plt.boxplot(spec_rs, labels = labels)
plt.show()

plt.figure()
plt.title('Photometric Redshift Across Classes')
plt.ylabel('Redshift')
plt.xlabel('Class')
plt.boxplot(photo_rs, labels = labels)
plt.show()

In [ ]:
#Lomb-Scargle Multiband Periodic Fit
#will extract features pertaining to periodicity
#from the light curves while incorporating the
#correlation of light in different passbands
#for unevenly sampled data

def fit_multiband(obj_id):
    #time, flux, flux error, and passband series
    t = train_series[train_series['object_id'] == obj_id]['mjd']
    f = train_series[train_series['object_id'] == obj_id]['flux']
    e = train_series[train_series['object_id'] == obj_id]['flux_err']
    b = train_series[train_series['object_id'] == obj_id]['passband']
    
    #parameterizing and fitting the model
    model = periodic.LombScargleMultibandFast(fit_period= True);
    
    #period range chosen on the assumption that
    #the period will be between 2.4 hours and half
    #of the observation window
    model.optimizer.period_range = (0.1, int((t.max()-t.min())/2));
    model.fit(t, f, e, b);
    
    #accuracy of fit for the model's best predicted period
    best_period_score = model.score(model.best_period);
    
    #returns model, best predicted period, and its fit score
    return [model, model.best_period, float(best_period_score)]

In [ ]:
#Discrete Wavelet Transform with PCA
#will help to extract dynamic characteristics
#of the data with relatively few features

#Smoothing spline regression technique to create
#evenly-spaced points since wavelet transform
#cannot accept two axes of information
def interpolate_signal(obj_id, band):
    
    #selecting time and flux data for the object
    t = (train_series.loc[train_series['object_id'] == obj_id][train_series['passband'] == band]['mjd'])
    f = (train_series.loc[train_series['object_id'] == obj_id][train_series['passband'] == band]['flux'])
    
    #cubic univariate spline function generation
    #where if loop handles case when there are 
    #three or less points
    if len(t) > 3:
        s0 = UnivariateSpline(t, f)
    elif len(t) == 3:
        s0 = UnivariateSpline(t, f, k=2)
    elif len(t) == 2:
        s0 = UnivariateSpline(t, f, k=1)
    elif len(t) == 1:
        s0 = UnivariateSpline(t, f, k=0)
    
    return s0, t.min(), t.max()

#discrete wavelet transform for light curve
#in single passband
def single_dwt(obj_id, band):
    #performing interpolation, selecting grid
    #in time (x) upon which to project
    interp, tmin, tmax = interpolate_signal(obj_id, band)
    x = np.linspace(tmin, tmax, 100) 
    y0 = interp(x)
    
    #padding the edges with constant values to
    #avoid edge effects from high-dimensional
    #wavelet transform
    y0 = pywt.pad(y0, 16, 'constant')
    
    #discrete wavelet transform on two levels using daubechies wavelet
    wav = pywt.wavedec(y0, wavelet = 'db1', level = 7)

    #concatenating the large-scale and small-scale frequencies detected
    wav_coefficients, wav_details = pywt.coeffs_to_array(wav)
    
    return wav_coefficients

#combine all wavelet transforms across all objects
def get_dynamic_feats():
    #create a new dataframe with ids
    dynamic_feats = pd.DataFrame(ids, columns = ['object_id'])
    
    #populate columns with dwt for each band
    dynamic_feats['dwt_0'] = dynamic_feats.apply(lambda row: single_dwt(row['object_id'], 0), axis = 1)
    dynamic_feats['dwt_1'] = dynamic_feats.apply(lambda row: single_dwt(row['object_id'], 1), axis = 1)
    dynamic_feats['dwt_2'] = dynamic_feats.apply(lambda row: single_dwt(row['object_id'], 2), axis = 1)
    dynamic_feats['dwt_3'] = dynamic_feats.apply(lambda row: single_dwt(row['object_id'], 3), axis = 1)
    dynamic_feats['dwt_4'] = dynamic_feats.apply(lambda row: single_dwt(row['object_id'], 4), axis = 1)
    dynamic_feats['dwt_5'] = dynamic_feats.apply(lambda row: single_dwt(row['object_id'], 5), axis = 1)
    
    del dynamic_feats['object_id']
    
    return dynamic_feats

#PCA to convert wavelet transform of a
#single band to a single feature
def covar_PCA(band):
    #reshaping the array so that
    #the covariance matrix can be applied
    oldwav = pd.DataFrame((untransformed['dwt_{}'.format(band)]).tolist())
    oldwav = oldwav.values
    m, n = oldwav.shape
    #centering the data around the mean
    #for each coefficient
    oldwav -= oldwav.mean(axis = 0)
    
    #calculating the covariance matrix
    covar = np.cov(oldwav, rowvar = False)
    #calculating the eigenvalues and eigenvectors
    #of the (symmetrical) covariance matrix
    evals, evecs = linalg.eigh(covar)
    
    #sorting eigenvalues in decreasing order
    #while keeping indices consistent between
    #evals and evecs
    idx = np.argsort(evals)[::-1]
    evecs = evecs[:, idx]
    evals = evals[idx]
    #selecting the most important eigenvector
    evecs = evecs[:, :1]
    
    newwav = np.dot(evecs.T, oldwav.T).T
    
    return newwav #newwav

In [ ]:
untransformed = get_dynamic_feats()
untransformed.head()

In [ ]:
train_metadata['wav_coeffs_0'] = covar_PCA(0)
train_metadata['wav_coeffs_1'] = covar_PCA(1)
train_metadata['wav_coeffs_2'] = covar_PCA(2)
train_metadata['wav_coeffs_3'] = covar_PCA(3)
train_metadata['wav_coeffs_4'] = covar_PCA(4)
train_metadata['wav_coeffs_5'] = covar_PCA(5)

In [ ]:
y = train_metadata['target']
train_metadata.head()

In [ ]:
#extracting features from timeseries

#garbage collector is useful as I manipulate
#timeseries and metadata to extract features
gc.enable() 

#extracting descriptive statistics about the time series
statistical_features = {
    'mjd': ['min', 'max', 'size'],
    'flux': ['min', 'max', 'mean', 'median', 'std'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'detected': ['mean']}

#simplifying the dataframe
train_feat = train_series.groupby(['object_id', 'passband']).agg(statistical_features)
new_columns = [k + '_' + agg for k in statistical_features.keys() for agg in statistical_features[k]]
train_feat.columns = new_columns

#considering the delta of time and flux
#for each object in each passband
train_feat['mjd_diff'] = train_feat['mjd_max']-train_feat['mjd_min']
train_feat['flux_diff'] = train_feat['flux_max']-train_feat['flux_min']
del train_feat['mjd_max'], train_feat['mjd_min']

gc.collect()

#making it so that each object is represented by one row
bands = [0, 1, 2, 3, 4, 5]

#creating the dataframe based on object id
rearranged = pd.DataFrame(ids, columns = ['object_id'])

for band in bands:
    banddata = train_feat.groupby(['passband']).get_group(band)

    rearranged['mjd_size_{}'.format(band)] = np.asarray(banddata['mjd_size'])
    rearranged['flux_min_{}'.format(band)] = np.asarray(banddata['flux_min'])
    rearranged['flux_max_{}'.format(band)] = np.asarray(banddata['flux_max'])
    rearranged['flux_mean_{}'.format(band)] = np.asarray(banddata['flux_mean'])
    rearranged['flux_median_{}'.format(band)] = np.asarray(banddata['flux_median'])
    rearranged['flux_std_{}'.format(band)] = np.asarray(banddata['flux_std'])
    rearranged['detected_mean_{}'.format(band)] = np.asarray(banddata['detected_mean'])
    rearranged['flux_err_min_{}'.format(band)] = np.asarray(banddata['flux_err_min'])
    rearranged['flux_err_max_{}'.format(band)] = np.asarray(banddata['flux_err_max'])
    rearranged['flux_err_mean_{}'.format(band)] = np.asarray(banddata['flux_err_mean'])
    rearranged['flux_err_median_{}'.format(band)] = np.asarray(banddata['flux_err_median'])
    rearranged['flux_err_std_{}'.format(band)] = np.asarray(banddata['flux_err_std'])
    rearranged['flux_err_skew_{}'.format(band)] = np.asarray(banddata['flux_err_skew'])
    rearranged['mjd_diff_{}'.format(band)] = np.asarray(banddata['mjd_diff'])
    rearranged['flux_diff_{}'.format(band)] = np.asarray(banddata['flux_diff'])

rearranged['target'] = y    

rearranged.head()

THERE ARE LARGE OBSERVATION GAPS DUE TO THE TELESCOPIC SETTINGS. SO THE DATA IS KIND OF SPARSE. BUT WE CAN TRY TO FOLD THE OBJECTS BY PERIOD BECAUSE SOME OBJECTS WILL EXHIBIT BEHAVIORS LIKE THIS. EXAMPLES???

NORMAL CLASSIFIERS ASSUME INDEPENDENT EXAMPLES. SINCE THIS IS TIME-SERIES DATA, POINTS THAT ARE CLOSE IN TIME WILL BE CORRELATED. 

In [ ]:
#define function to create raw light curve plot
def raw_curve(obj_id):
    
    obj_class = int(train_metadata['target'][train_metadata['object_id'] == obj_id])
    
    u = mpatches.Patch(color = 'red', label = 'u')
    g = mpatches.Patch(color = 'orange', label = 'g')
    r = mpatches.Patch(color = 'yellow', label = 'r')
    i = mpatches.Patch(color = 'green', label = 'i')
    z = mpatches.Patch(color = 'blue', label = 'z')
    y = mpatches.Patch(color = 'purple', label = 'y')
    
    plt.figure()
    plt.legend(handles=[u, g, r, i, z, y])
    plt.title('Raw Light Curve: Object {}'.format(obj_id)+ '; Class {}'.format(obj_class))
    plt.xlabel('mjd')
    plt.ylabel('flux')

    plt.scatter(x = np.array(train_metadata['mjd_passband_0'][train_metadata['object_id'] == obj_id])[0], y = np.array(train_metadata['flux_passband_0'][train_metadata['object_id'] == obj_id])[0], color='red')
    plt.scatter(x = np.array(train_metadata['mjd_passband_1'][train_metadata['object_id'] == obj_id])[0], y = np.array(train_metadata['flux_passband_1'][train_metadata['object_id'] == obj_id])[0], color='orange')
    plt.scatter(x = np.array(train_metadata['mjd_passband_2'][train_metadata['object_id'] == obj_id])[0], y = np.array(train_metadata['flux_passband_2'][train_metadata['object_id'] == obj_id])[0], color='yellow')
    plt.scatter(x = np.array(train_metadata['mjd_passband_3'][train_metadata['object_id'] == obj_id])[0], y = np.array(train_metadata['flux_passband_3'][train_metadata['object_id'] == obj_id])[0], color='green')
    plt.scatter(x = np.array(train_metadata['mjd_passband_4'][train_metadata['object_id'] == obj_id])[0], y = np.array(train_metadata['flux_passband_4'][train_metadata['object_id'] == obj_id])[0], color='blue')
    plt.scatter(x = np.array(train_metadata['mjd_passband_5'][train_metadata['object_id'] == obj_id])[0], y = np.array(train_metadata['flux_passband_5'][train_metadata['object_id'] == obj_id])[0], color='purple')
    
    return

#define function to print phase plot & best periodic fit
def phase_curve(obj_id, model, best_period, best_score):
    
    obj_class = int(train_metadata['target'][train_metadata['object_id'] == obj_id])
    
    #single phase in each passband
    plt.figure()
    plt.title('Phase Plot: Object {}'.format(obj_id) + ': Class {}'.format(obj_class))
    plt.xlabel('phase')
    plt.ylabel('relative flux')
    plt.scatter(x = np.array(train_metadata['mjd_passband_0'][train_metadata['object_id'] == obj_id])[0] / (model.best_period) % 1, y = np.array(train_metadata['flux_passband_0'][train_metadata['object_id'] == obj_id])[0], color = 'red')
    plt.scatter(x = np.array(train_metadata['mjd_passband_1'][train_metadata['object_id'] == obj_id])[0] / (model.best_period) % 1, y = np.array(train_metadata['flux_passband_1'][train_metadata['object_id'] == obj_id])[0], color = 'orange')
    plt.scatter(x = np.array(train_metadata['mjd_passband_2'][train_metadata['object_id'] == obj_id])[0] / (model.best_period) % 1, y = np.array(train_metadata['flux_passband_2'][train_metadata['object_id'] == obj_id])[0], color = 'yellow')
    plt.scatter(x = np.array(train_metadata['mjd_passband_3'][train_metadata['object_id'] == obj_id])[0] / (model.best_period) % 1, y = np.array(train_metadata['flux_passband_3'][train_metadata['object_id'] == obj_id])[0], color = 'green')
    plt.scatter(x = np.array(train_metadata['mjd_passband_4'][train_metadata['object_id'] == obj_id])[0] / (model.best_period) % 1, y = np.array(train_metadata['flux_passband_4'][train_metadata['object_id'] == obj_id])[0], color = 'blue')
    plt.scatter(x = np.array(train_metadata['mjd_passband_5'][train_metadata['object_id'] == obj_id])[0] / (model.best_period) % 1, y = np.array(train_metadata['flux_passband_5'][train_metadata['object_id'] == obj_id])[0], color = 'purple')
    
    #best periodic fit for each passband
    plt.figure()
    plt.title('Best Periodic Fit: {}'.format(best_period) + '; Period Score: {}'.format(best_score))
    plt.xlabel('phase')
    plt.ylabel('relative flux')
    yfit = model.predict(np.linspace(0, best_period, 1000), 0)
    plt.plot(np.linspace(0, best_period, 1000), yfit, color = 'red')
    yfit = model.predict(np.linspace(0, best_period, 1000), 1)
    plt.plot(np.linspace(0, best_period, 1000), yfit, color = 'orange')
    yfit = model.predict(np.linspace(0, best_period, 1000), 2)
    plt.plot(np.linspace(0, best_period, 1000), yfit, color = 'yellow')
    yfit = model.predict(np.linspace(0, best_period, 1000), 3)
    plt.plot(np.linspace(0, best_period, 1000), yfit, color = 'green')
    yfit = model.predict(np.linspace(0, best_period, 1000), 4)
    plt.plot(np.linspace(0, best_period, 1000), yfit, color = 'blue')
    yfit = model.predict(np.linspace(0, best_period, 1000), 5)
    plt.plot(np.linspace(0, best_period, 1000), yfit, color = 'purple')
    
    return


In [ ]:
#function to generate plots and features for a single object

def analyze_characteristics(object_id):
    raw_curve(object_id)
    [model, best_period, best_score] = fit_multiband(object_id)
    phase_curve(object_id, model, best_period, best_score)
    
    return [best_period, best_score]
    
#calculating the average and standard deviation
#of each of the statistical time series features
#where is flux typically and how does it vary for
#each of these objects? 
#def ts_by_target(target):
#    target_pd = rearranged[rearranged['target'] == target]
#    del target_pd['object_id'], target_pd['target']
#    feats = target_pd.columns
#    avgs = np.array(target_pd.mean(axis = 0))
#    stds = np.array(target_pd.std(axis = 0))
#    
#    df = pd.DataFrame(columns = feats)
#    df.loc['avg'] = avgs
#    df.loc['std'] = stds
#    
#    return df
    
#calculating the average and stardard deviation
#of each of the wavelet coefficients
def wav_by_target(target):
    target_pd = (train_metadata[train_metadata['target'] == target]).iloc[:, -6:]
    feats = target_pd.columns
    avgs = np.array(target_pd.mean(axis = 0))
    stds = np.array(target_pd.std(axis=0))
    
    df = pd.DataFrame(columns = feats)
    df.loc['avg'] = avgs
    df.loc['std'] = stds
    
    return df

In [ ]:
def class_analysis(target):
    
    metaseries = train_metadata.loc[train_metadata['target'] == target]
    ids = (metaseries['object_id'].unique())
    
    #generate first four examples using the object analysis
    analyze_characteristics(ids[0])
    analyze_characteristics(ids[1])
    analyze_characteristics(ids[2])
    analyze_characteristics(ids[3])
    
    #plot distribution of detected == 1 events per object
    #detected == 1 means that the signal is significantly
    #different than the background flux
    #using same loop to plot distribution of best period
    best_periods = []
    best_scores = []
    detecteds = []
    
    for x in ids:
        xseries = train_series[(train_series['object_id'] == x)]
        detectedx = xseries[['detected']]
        detecteds += [int(detectedx.sum())]
        model, best_period, best_score = fit_multiband(x)
        best_periods += [best_period]
        best_scores += [best_score]
        
    
    #create histogram from detecteds
    plt.figure()
    plt.hist(detecteds)
    plt.title('Frequency of Detected Events: Class {}'.format(target))
    plt.xlabel('Number of Detected Events')
    plt.ylabel('Frequency')
    
    #create histogram from best periods
    plt.figure()
    plt.hist(best_periods)
    plt.title('Distribution of Best Period: Class {}'.format(target))
    plt.xlabel('Period')
    plt.ylabel('Occurrences')
    avg_best_score = np.mean(best_scores)
    
    #naive benchmark for whether events tend to occur within or beyond our galaxy
    hostgal = metaseries[['hostgal_specz']]
    
    #check if 'hostgal_specz' has any zero component
        #if so, such events can occur within galaxy
    within_galaxy_possible = (0 in hostgal.values)
    #check if 'hostgal_specz' has all zero components
        #if so, such events exclusively occur within the galaxy
    within_galaxy_must = (hostgal.values == 0).all()
    
    #ts_analysis = ts_by_target(target)
    wav_analysis = wav_by_target(target)
    
    return (wav_analysis, within_galaxy_possible, within_galaxy_must, avg_best_score)

In [ ]:
class_analysis(90)

In [ ]:
class_analysis(42)

In [ ]:
class_analysis(65)

In [ ]:
class_analysis(16)

In [ ]:
class_analysis(15)

In [ ]:
class_analysis(62)

In [ ]:
class_analysis(88)

In [ ]:
class_analysis(92)

In [ ]:
class_analysis(67)

In [ ]:
class_analysis(52)

In [ ]:
class_analysis(95)

In [ ]:
class_analysis(6)

In [ ]:
class_analysis(64)

In [ ]:
class_analysis(53)